# Question 2: CMS Account - Savannah Sew-Hee
The CMS collaboration has an automated job submission system that runs jobs as "cmslocal" and "cmspilot".

For these two users, jobs have internal system tests that will terminate their jobs early after approximately 30 minutes.

Do any of their jobs that ended in under an hour also cluster on specific compute nodes, 
suggesting possbily unreliable systems? 

Check both “production” and “nogpfs” partitions. 

Look for commonly failing nodes and compare with other failed jobs.
_____________________________________________________________________________

In [ ]:
import numpy as np 
import pandas as pd 
import re
from io import StringIO
from datetime import time

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)

In [ ]:
# reformat the CSV to use | instead of , to separate rows, then import [a selection] of the data as a dataframe
for_pd = StringIO()
with open('../data/accre-jobs-2020.csv') as accre:
    for line in accre:
        new_line = re.sub(r',', '|', line.rstrip(), count=12)
        print (new_line, file=for_pd)

for_pd.seek(0)

accre_df = pd.read_csv(for_pd, sep='|')#[1000000:1005000] # add this to subset

# Cleaning accre_df

#### Converting time to seconds

In [ ]:
#making column for hours:min:sec
accre_df['hours_min_sec_req'] = accre_df['REQTIME'].str[-8:]
accre_df['hours_min_sec_used'] = accre_df['USEDTIME'].str[-8:]

#making column for days
accre_df['day_req'] = accre_df['REQTIME'].str.extract('(.*?)-')
accre_df['day_used'] = accre_df['USEDTIME'].str.extract('(.*?)-')

#filling null rows with zeros
accre_df['day_req'] = accre_df['day_req'].fillna(0)
accre_df['day_used'] = accre_df['day_used'].fillna(0)

# converting to the correct type
accre_df['day_req'] = accre_df['day_req'].astype(int)
accre_df['day_used'] = accre_df['day_used'].astype(int)

In [ ]:
#converting to timedelta to then use total_seconds
accre_df['hours_min_sec_req'] =  pd.to_timedelta(accre_df['hours_min_sec_req'], unit='s')
accre_df['hours_min_sec_used'] =  pd.to_timedelta(accre_df['hours_min_sec_used'], unit='s')
accre_df['hours_min_sec_req'] = accre_df['hours_min_sec_req'].dt.total_seconds()
accre_df['hours_min_sec_used'] = accre_df['hours_min_sec_used'].dt.total_seconds()

In [ ]:
#making total seconds columns 
accre_df['total_sec_req'] = (accre_df['day_req'] * 86400) + accre_df['hours_min_sec_req']
accre_df['total_sec_used'] = (accre_df['day_used'] * 86400) + accre_df['hours_min_sec_used']

# Cleaning CMS Account Data

In [ ]:
#subsetting data for cms account
cms_df = accre_df[accre_df['ACCOUNT']=='cms']

#subsetting data to get rid of debug partition
cms_df = cms_df[cms_df['PARTITION']!='debug']

#subsetting data for only 'cmspilot' and 'cmslocal' users
cms_df = cms_df[(cms_df['USER']=='cmspilot')|(cms_df['USER']=='cmslocal')]

#printing how many CMS jobs including over an hour
print(cms_df.shape)

#subsetting for time under an hour
cms_df = cms_df[cms_df['total_sec_used'] <= 3600]

#getting only failed jobs
#using state != to complete instead of exit code beacuse its built in their code to crash and error might not be reflected in exit code
cms_failed = cms_df[cms_df['STATE'] != 'COMPLETED']
cms_completed = cms_df[cms_df['STATE'] == 'COMPLETED']

# EDA and Analysis

In [ ]:
#Examining the difference between a 'COMPLETED' job and exitcode '0:0'
pd.crosstab(cms_df['EXITCODE'],cms_df['STATE']).apply(lambda x: (x/x.sum()), axis=1)

### How often are jobs canceling around 30 minutes for account CMS?

In [ ]:
#Looking at distribution of seconds to find spike
#where jobs are canceling sround 30 min
cms_df.hist(column = 'total_sec_used')

In [ ]:
#How many jobs ended between 500 and 2000 seconds
#or 1000 and 1500 use this one second spike
cms_jobs_ended_around_30_min = cms_df[(cms_df['total_sec_used'] < 1500) & (cms_df['total_sec_used'] > 1000)]

print("Number of CMS Jobs Total:")
print("(699831, 19)")
print("Number of CMS Jobs Under Hour:")
print(cms_df.shape)
print("Number of CMS Jobs Ended Around 30 Minutes:")
print(cms_jobs_ended_around_30_min.shape)

In [ ]:
#what percent of jobs is this?
print(str(round((263645/699831)*100,2)) + "% are ending around 30 minutes\nout of total CMS jobs (cmspilot/cmslocal users)")
print("\n")
print(str(round((263645/447255)*100,2)) + "% are ending around 30 minutes\nout of jobs ending in less than an hour")

### Most Failed Nodes for CMS Account

In [ ]:
most_failed_nodes = cms_df['NODELIST'].value_counts().head(20)

In [ ]:
ax = most_failed_nodes.plot(kind = 'bar', figsize = (8,6))
ax.set_title('CMS Account: Nodes that Failed the Most',weight='bold', size='large')
ax.set_xlabel('Node')
ax.set_ylabel('Number of Time Failed')

In [ ]:
most_failed_nodes.head(10)

### Percent of Times that Nodes Failed for CMS Account

In [ ]:
#Count of failed nodes for new df for percents
cms_failed['NODELIST'].value_counts().head()

In [ ]:
#Count of successful nodes for new df for percents
cms_completed['NODELIST'].value_counts().head()

In [ ]:
#Creating df with top nodes to examine by percent of times failed
CMS_nodes = {'NODE':['ng518','cn1314','cn394','ng734','cn475','cn1094','ng1112','cn449','cn1121','cn304','cn1394','cn408','cn1387','cn399','cn363','cn429','cn1398','cn312'],
            'TIMES_FAILED':[16,12,11,10,9,9,9,9,9,9,8,8,8,8,8,8,8,8],
            'TOTAL_TIMES_USED':[19352,12,347,4138,364,397,6171,400,465,298,408,446,249,415,339,306,550,407]}
CMS_nodes_df = pd.DataFrame(CMS_nodes, columns = ['NODE','TIMES_FAILED','TOTAL_TIMES_USED'])

In [ ]:
CMS_nodes_df['PERCENT_FAILED'] = (CMS_nodes_df['TIMES_FAILED']/CMS_nodes_df['TOTAL_TIMES_USED'])*100

In [ ]:
#plotted with one outlier, maybe remove it to make it easier to see
ax = CMS_nodes_df.sort_values('PERCENT_FAILED').plot.bar(x = 'NODE', y = 'PERCENT_FAILED', figsize = (8,6))
ax.set_title('Percent Node Failed for CMS Account',weight='bold', size='large')
ax.set_xlabel('Node')
ax.set_ylabel('Percent of Times Failed')

In [ ]:
#removed cn 1314 to make it to make it easier to see,  label this graph
CMS_nodes_without_cn1314 = CMS_nodes_df[CMS_nodes_df['NODE'] != 'cn1314']
ax = CMS_nodes_without_cn1314.sort_values('PERCENT_FAILED').plot.bar(x = 'NODE', y = 'PERCENT_FAILED',figsize = (8,6))
ax.set_title('Percent Node Failed for CMS Account (without Outlier CN1314)',weight='bold', size='large')
ax.set_xlabel('Node')
ax.set_ylabel('Percent of Times Failed')

In [ ]:
CMS_nodes_df.sort_values('PERCENT_FAILED', ascending = False).head()

# Question 4

In [ ]:
# remove unneeded columns
accre = accre_df.drop(["JOBID","USER", "NODELIST"], axis =1) # axis = 1 so that it works across our columns

In [ ]:
#new column is for requested memory per node
accre['RMPN'] = accre['REQMEM'].str.extract('(.*)Mn$')

#new column is for requested memory per core
accre['RMPC'] = accre['REQMEM'].str.extract('(.*)Mc$')

#new column is for requested memory per core
accre['UM'] = accre['USEDMEM'].str.extract('(.*)M$')

In [ ]:
# change RMPC,RMPN,UM colum to fill with na with 0
accre['RMPC'] = accre['RMPC'].fillna('0')
accre['RMPN'] = accre['RMPN'].fillna('0')
accre['UM'] = accre['UM'].fillna('0')

In [ ]:
#fixing types
accre['RMPC']= accre['RMPC'].astype(str).astype(float)
accre['RMPN']= accre['RMPN'].astype(str).astype(float)
accre['UM']= accre['UM'].astype(str).astype(float)

In [ ]:
# what is per core??? assume you multiply cpus by node this column will be RMPN times CPUS 
#3.5 is the average node to core ratio
accre['RMPN'] = accre['RMPN'] /(accre['CPUS']/ accre['NODES'])

In [ ]:
# what is per core??? update RMPC to add the RMPNEW column
accre['RMPC'] = accre['RMPC'] + accre['RMPN']

# remove unneeded columns
accre = accre.drop(["RMPN"], axis =1)

In [ ]:
#convert used memory to cores 
accre['UM'] = (accre['UM'] /accre['CPUS'])/ accre['NODES']

In [ ]:
# PERCENT of request to actual used
accre['PRU'] = (accre['UM']/accre['RMPC']) * 100

### Weighted Average

In [ ]:
#creating a weighted average function
def wavg(group, avg_name, weight_name):
    """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    In rare instance, we may not have weights, so just return the mean. Customize this if your business case
    should return otherwise.
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [ ]:
#weighted average for all acounts
wavg(accre, "PRU", "total_sec_used")

In [ ]:
#weighted average by account (best Optimizing to least)
weighted_mem_accre_accounts = accre.groupby("ACCOUNT").apply(wavg, "PRU", "total_sec_used").sort_values(ascending = False)

In [ ]:
# top 10 at optimizing memory
weighted_mem_accre_accounts.head(10)

In [ ]:
# bottom 10 at optimizing memory
weighted_mem_accre_accounts.tail(10)